In [7]:
%pip install ta-lib

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build ta-lib
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Building wheel for ta-lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [28 lines of output]
      <string>:75: UserWarning: Cannot find ta-lib library, installation may fail.
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-312\talib
      copying talib\abstract.py -> build\lib.win-amd64-cpython-312\talib
      copying talib\deprecated.py -> build\lib.win-amd64-cpython-312\talib
      copying talib\stream.py -> build\lib.win-amd64-cpython-312\talib
      copying talib\__init__.py -> build\lib.win-amd64-cpython-312\talib
      running egg_info
      writing ta_lib.egg-info\PKG-INFO
      writing dependency_links to ta_lib.egg-info\dependency_links.txt
      writing requirements to ta_lib.egg-info\requires.txt
      writing top-level names to ta_lib.egg-info\top_level.txt
      reading manifest file 'ta_lib.egg-info\SOURCES.txt'
      reading manifest template 

In [15]:
import talib
import talib
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
import yfinance as yf
import plotly.express as px
from sklearn.linear_model import LinearRegression
from datetime import datetime, timedelta
import backtesting
from backtesting import Backtest, Strategy
import warnings
import itertools

In [4]:
# загрузим данные BTC
df = yf.download("BTC-USD", start='2023-01-01', end='2024-08-02', interval='1d').drop(columns=['Adj Close']) #'2024-01-02'
df = df.reset_index()
df.columns = df.columns.str.lower()
df.head()

[*********************100%%**********************]  1 of 1 completed


,date,open,high,low,close,volume
0,2023-01-01,16547.914062,16630.439453,16521.234375,16625.080078,9244361700
1,2023-01-02,16625.509766,16759.343750,16572.228516,16688.470703,12097775227
2,2023-01-03,16688.847656,16760.447266,16622.371094,16679.857422,13903079207
3,2023-01-04,16680.205078,16964.585938,16667.763672,16863.238281,18421743322
4,2023-01-05,16863.472656,16884.021484,16790.283203,16836.736328,13692758566


In [5]:
class TemaMacdStrategy(Strategy):
    
    def init(self):
        self.signal = self.I(lambda: self.data.Signal)
        self.previous_signal = 0
        self.size = 0.1

    def next(self):
        current_signal = self.signal[-1]

        if current_signal != self.previous_signal:
            if current_signal == 1:
                if self.position.is_short:
                    self.position.close()
                    
                if not self.position.is_long:
                    self.buy(size=self.size)
                    
            elif current_signal == -1:
                if self.position.is_long:
                    self.position.close()
                   
                if not self.position.is_short:
                    self.sell(size=self.size)
                    
            elif current_signal == 0:
                if self.position:
                    self.position.close()

        self.previous_signal = current_signal

### Walk Forward Optimization

In [6]:
# напиешм функию стартегии
def apply_strategy(data, params):
    """
    Применение стратегии с оптимизированными параметрами.

    :param df: DataFrame с данными, на которые будут наложены индикаторы.
    :param params: Словарь с оптимизированными параметрами.
    :return: DataFrame с рассчитанными индикаторами и сигналами.
    """
    df = data.copy()
    # Извлекаем параметры из словаря
    tema_period = params['tema_period']
    fastMACD_period = params['fastMACD_period']
    slowMACD_period = params['slowMACD_period']
    signalMACD_period = params['signalMACD_period']
    
    # Добавляем индикаторы
    df['tema'] = talib.TEMA(df['close'], timeperiod=tema_period)
    df['macd'], df['macd_signal'], df['macd_hist'] = talib.MACD(df['close'], fastperiod=fastMACD_period, slowperiod=slowMACD_period, signalperiod=signalMACD_period)
    
    
    # Создаем сигналы для покупки и продажи
    df['signal'] = 0
    df.loc[(df['macd'] > df['macd_signal']) & (df['close'] > df['tema']), 'signal'] = 1  # Сигнал на покупку
    df.loc[(df['macd'] < df['macd_signal']) & (df['close'] < df['tema']), 'signal'] = -1  # Сигнал на продажу
    
    return df[["date", "open", "high", "low", "close", "volume","signal"]]

def bactest_strategy(df, strategy_class, params, plot=False):
    """
    Запускает бэктест с переданными параметрами стратегии.

    :param df: DataFrame с данными для бэктеста.
    :param strategy_class: Класс стратегии для бэктеста.
    :param params: Словарь с параметрами стратегии.
    :return: Статистика бэктеста.
    """
    # Применяем стратегию с переданными параметрами
    df = apply_strategy(df.copy(), params)
    
    # Подготовка данных для бэктеста
    bt_df = df.copy()
    bt_df.columns = bt_df.columns.str.capitalize()
    bt_df.rename(columns={'Date': 'Datetime'}, inplace=True)
    bt_df["Datetime"] = pd.to_datetime(bt_df["Datetime"])
    bt_df.set_index('Datetime', inplace=True)

    # Создаем объект класса Backtest с текущей стратегией
    bt = Backtest(bt_df, strategy_class, cash=500000, commission=.002, exclusive_orders=True, margin=0.1)

    # Запускаем бэктест
    stats = bt.run()
    if plot:
        bt.plot(
        plot_equity=True,
        plot_drawdown=True,
        relative_equity=False,
        )
    return stats

def get_best_strategy(buffer, strategy_class):
    # Задаем возможные значения для параметров стратегии
    tema_period_list = [7, 14, 28]
    fastMACD_period_list = [12, 35, 56]
    slowMACD_period_list = [9,23, 39]
    signalMACD_period_list = [28,40,80]

    # Для хранения лучших параметров и лучшего результата
    best_params = None
    best_performance = -float('inf')  # или другое значение, которое имеет смысл для вашей метрики

    # Проходим по всем комбинациям параметров
    for tema_period, fastMACD_period, slowMACD_period, signalMACD_period in itertools.product(tema_period_list, fastMACD_period_list, slowMACD_period_list, signalMACD_period_list):
        
        # Создаем словарь с текущими параметрами
        params = {
            'tema_period': tema_period,
            'fastMACD_period': fastMACD_period,
            'slowMACD_period': slowMACD_period,
            'signalMACD_period': signalMACD_period
        }
        
        # Запускаем бэктест с текущими параметрами
        stats = bactest_strategy(buffer.copy(), strategy_class, params)

        # Определяем метрику, по которой будем выбирать лучшую стратегию (например, по профит фактору)
        performance = stats['Profit Factor']  # используйте другую метрику, если нужно
        
        # Сравниваем с лучшим результатом и сохраняем лучшие параметры
        if performance > best_performance:
            best_performance = performance
            best_params = params

    print(f"Best Performance: {best_performance}")
    print(f"Best Parameters: {best_params}")
    return best_params

    

In [7]:

train_size = 90  # Размер окна тренировки
test_size = 30    # Размер тестового окна

# Инициализация DataFrame для сигналов
signals_df = pd.DataFrame()

# Определение количества итераций
num_iterations = (len(df) - train_size) // test_size

for i in range(num_iterations + 1):
    # Определение границ обучающего и тестового окон
    start_train = i * test_size
    end_train = start_train + train_size
    start_test = end_train
    end_test = start_test + test_size
    
    # Если конец тестового окна выходит за пределы данных, обрезаем его
    if end_test > len(df):
        end_test = len(df)
    
    # Определяем окна для тренировки и тестирования
    train_data = df.iloc[start_train:end_train].copy()
    test_data = df.iloc[start_test:end_test].copy()
    
    
    # Оптимизация на тренировочном окне
    best_params = get_best_strategy(train_data, TemaMacdStrategy)
    
    # Объединяем данные тренировки и теста
    combined_data = pd.concat([train_data, test_data]).reset_index(drop=True)
    
    # Применяем стратегию на объединенном окне с оптимальными параметрами
    combined_with_signal = apply_strategy(combined_data.copy(), best_params)
    
    # Извлекаем только часть данных, относящуюся к тестовому окну
    test_with_signal = combined_with_signal.iloc[-test_size:].copy()
    
    # Добавляем сигналы из тестового окна в signals_df
    signals_df = pd.concat([signals_df, test_with_signal], ignore_index=True)




c:\Users\ma2se\OneDrive\Documents\VS_CODE_PROJECTS\notebooks\.venv\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


Best Performance: 43.223250492662174
Best Parameters: {'tema_period': 14, 'fastMACD_period': 12, 'slowMACD_period': 9, 'signalMACD_period': 40}
Best Performance: 24.585737563930497
Best Parameters: {'tema_period': 14, 'fastMACD_period': 12, 'slowMACD_period': 9, 'signalMACD_period': 28}
Best Performance: 3.929440297636021
Best Parameters: {'tema_period': 14, 'fastMACD_period': 12, 'slowMACD_period': 23, 'signalMACD_period': 40}
Best Performance: 7.71696338775185
Best Parameters: {'tema_period': 14, 'fastMACD_period': 35, 'slowMACD_period': 9, 'signalMACD_period': 40}
Best Performance: 8.727510527093663
Best Parameters: {'tema_period': 7, 'fastMACD_period': 12, 'slowMACD_period': 9, 'signalMACD_period': 40}
Best Performance: 145.67697803922576
Best Parameters: {'tema_period': 7, 'fastMACD_period': 12, 'slowMACD_period': 39, 'signalMACD_period': 28}
Best Performance: 9.452110508881162
Best Parameters: {'tema_period': 7, 'fastMACD_period': 35, 'slowMACD_period': 9, 'signalMACD_period': 28

c:\Users\ma2se\OneDrive\Documents\VS_CODE_PROJECTS\notebooks\.venv\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\ma2se\OneDrive\Documents\VS_CODE_PROJECTS\notebooks\.venv\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\ma2se\OneDrive\Documents\VS_CODE_PROJECTS\notebooks\.venv\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0

Best Performance: 145.04999678444383
Best Parameters: {'tema_period': 7, 'fastMACD_period': 12, 'slowMACD_period': 23, 'signalMACD_period': 40}
Best Performance: 1.5153091577837519
Best Parameters: {'tema_period': 14, 'fastMACD_period': 12, 'slowMACD_period': 23, 'signalMACD_period': 28}
Best Performance: 0.7149819641154821
Best Parameters: {'tema_period': 7, 'fastMACD_period': 12, 'slowMACD_period': 9, 'signalMACD_period': 40}
Best Performance: 5.629139802494213
Best Parameters: {'tema_period': 14, 'fastMACD_period': 35, 'slowMACD_period': 23, 'signalMACD_period': 40}


c:\Users\ma2se\OneDrive\Documents\VS_CODE_PROJECTS\notebooks\.venv\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\ma2se\OneDrive\Documents\VS_CODE_PROJECTS\notebooks\.venv\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501
c:\Users\ma2se\OneDrive\Documents\VS_CODE_PROJECTS\notebooks\.venv\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0

...

Best Performance: 1.3660265206033158
Best Parameters: {'tema_period': 14, 'fastMACD_period': 12, 'slowMACD_period': 9, 'signalMACD_period': 28}
Best Performance: 2.9785312974475358
Best Parameters: {'tema_period': 14, 'fastMACD_period': 12, 'slowMACD_period': 23, 'signalMACD_period': 40}


c:\Users\ma2se\OneDrive\Documents\VS_CODE_PROJECTS\notebooks\.venv\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


Best Performance: 0.5038340864383556
Best Parameters: {'tema_period': 7, 'fastMACD_period': 12, 'slowMACD_period': 9, 'signalMACD_period': 28}
Best Performance: 1.860710773044403
Best Parameters: {'tema_period': 14, 'fastMACD_period': 35, 'slowMACD_period': 9, 'signalMACD_period': 28}


c:\Users\ma2se\OneDrive\Documents\VS_CODE_PROJECTS\notebooks\.venv\Lib\site-packages\backtesting\_stats.py:122: RuntimeWarning: divide by zero encountered in scalar divide
  s.loc['Sortino Ratio'] = np.clip((annualized_return - risk_free_rate) / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


Best Performance: 5.784897882215679
Best Parameters: {'tema_period': 7, 'fastMACD_period': 12, 'slowMACD_period': 9, 'signalMACD_period': 28}


In [8]:

bt_df = signals_df.copy()
bt_df.columns = bt_df.columns.str.capitalize()
bt_df.rename(columns={'Date': 'Datetime'}, inplace=True)
bt_df["Datetime"] = pd.to_datetime(bt_df["Datetime"])
bt_df.set_index('Datetime', inplace=True)
bt_df = bt_df.sort_index()

bt = Backtest(bt_df, TemaMacdStrategy, cash=500000, commission=0.002, exclusive_orders=True, margin=0.1)

    # Запускаем бэктест
stats = bt.run()
print(stats[:27])
bt.plot(
plot_equity=True,
plot_drawdown=True,
relative_equity=False,
)

Start                     2023-04-01 00:00:00
End                       2024-08-01 00:00:00
Duration                    488 days 00:00:00
Exposure Time [%]                   58.627451
Equity Final [$]                609247.716012
Equity Peak [$]                 754682.093559
Return [%]                          21.849543
Buy & Hold Return [%]              130.042656
Return (Ann.) [%]                   15.893956
Volatility (Ann.) [%]               36.921833
Sharpe Ratio                         0.430476
Sortino Ratio                        0.795714
Calmar Ratio                         0.422703
Max. Drawdown [%]                  -37.600725
Avg. Drawdown [%]                   -5.721549
Max. Drawdown Duration      265 days 00:00:00
Avg. Drawdown Duration       36 days 00:00:00
# Trades                                   84
Win Rate [%]                         40.47619
Best Trade [%]                      13.551881
Worst Trade [%]                     -7.854729
Avg. Trade [%]                    

c:\Users\ma2se\OneDrive\Documents\VS_CODE_PROJECTS\notebooks\.venv\Lib\site-packages\backtesting\_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


Row(id='1524', ...)

### GBM    https://ru.wikipedia.org/wiki/%D0%93%D0%B5%D0%BE%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%BE%D0%B5_%D0%B1%D1%80%D0%BE%D1%83%D0%BD%D0%BE%D0%B2%D1%81%D0%BA%D0%BE%D0%B5_%D0%B4%D0%B2%D0%B8%D0%B6%D0%B5%D0%BD%D0%B8%D0%B5

In [13]:
def generate_log_return_scenarios(df, num_scenarios):
    scenarios = []
    
    # Вычисляем логарифмические приросты для исходного столбца Close
    log_returns = np.log(df['close'] / df['close'].shift(1)).dropna()
    
    for _ in range(num_scenarios):
        scenario_df = df.copy()
        
        # Перемешиваем логарифмические приросты
        shuffled_log_returns = np.random.permutation(log_returns.values)
        
        # Генерируем новый столбец Close, начиная с начальной цены
        new_close = [df['close'].iloc[0]]  # начальная цена остается неизменной
        for log_return in shuffled_log_returns:
            new_close.append(new_close[-1] * np.exp(log_return))
        
        # Вставляем NaN в начало, чтобы выровнять длину с исходным DataFrame
        new_close_series = pd.Series(new_close, index=df.index)
        new_close_series.iloc[1:] = new_close_series.iloc[1:]
        
        # Записываем сгенерированные значения в новый столбец Close
        scenario_df['close'] = new_close_series
        
        # Добавляем сгенерированный DataFrame в список сценариев
        scenarios.append(scenario_df)
    
    return scenarios



train_size = 0.8  # 80% данных для тренировки
train_df = df.iloc[:int(len(df) * train_size)]
test_df = df.iloc[int(len(df) * train_size):]
num_scenarios = 5
scenarios = generate_log_return_scenarios(train_df, num_scenarios)

In [14]:
fig = go.Figure()

# Добавляем оригинальную цену
fig.add_trace(go.Scatter(x=train_df['date'], y=train_df['close'],
                         mode='lines', name='Original Close',
                         line=dict(color='red')))

# Добавляем сгенерированные сценарии
for i, scenario_df in enumerate(scenarios):
    fig.add_trace(go.Scatter(x=scenario_df['date'], y=scenario_df['close'],
                             mode='lines', name=f'Scenario {i+1}',
                             line=dict(width=1, color='blue')))

# Настройка осей и названий
fig.update_layout(title='Original vs Generated Scenarios',
                  xaxis_title='Date',
                  yaxis_title='Close Price',
                  template='plotly_dark')

# Отображаем график
fig.show()

In [11]:
stats_list = [
    'Return [%]',
    'Max. Drawdown [%]',
    'Sharpe Ratio',
    #'Avg Win [%]',
    #'Avg Loss [%]'
]
all_stats = []
for scenario_df in scenarios:
    scenario_df['tema'] = talib.TEMA(df['close'], timeperiod=24)
    scenario_df['macd'], scenario_df['macd_signal'], scenario_df['macd_hist'] = talib.MACD(scenario_df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    # Создаем сигналы для покупки и продажи
    scenario_df['signal'] = 0
    scenario_df.loc[(scenario_df['macd'] > scenario_df['macd_signal']) & (scenario_df['close'] > scenario_df['tema']), 'signal'] = 1  # Сигнал на покупку
    scenario_df.loc[(scenario_df['macd'] < scenario_df['macd_signal']) & (scenario_df['close'] < scenario_df['tema']), 'signal'] = -1  # Сигнал на продажу

    # бектестим
    bt_df = scenario_df.copy()
    bt_df.columns = bt_df.columns.str.capitalize()
    bt_df.rename(columns={'Date': 'Datetime'}, inplace=True)
    bt_df["Datetime"] = pd.to_datetime(bt_df["Datetime"])
    bt_df.set_index('Datetime', inplace=True)

    # Создаем объект класса Backtest
    bt = Backtest(bt_df, TemaMacdStrategy, cash=500000, commission=0.002, exclusive_orders=True, margin=0.1)

    # Запускаем бэктест
    stats = bt.run()
    all_stats.append(stats)

    
    

In [223]:
for i, stat in enumerate(stats_list):
        data = [stat_item[stat] for stat_item in all_stats]
        fig = go.Figure()
        fig.add_trace(go.Histogram(x=data,  nbinsx=20, name=stat))
    
        fig.update_layout(showlegend=False, title_text=f"Distributions of Backtest {stat}")
        fig.show()

### Random Walk https://ru.wikipedia.org/wiki/%D0%A1%D0%BB%D1%83%D1%87%D0%B0%D0%B9%D0%BD%D0%BE%D0%B5_%D0%B1%D0%BB%D1%83%D0%B6%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5

In [12]:
def generate_diff_scenarios(df, n_scenarios, length):
    # Вычисляем простые приросты (разности)
    diff_returns = df["close"].diff().dropna()
    
    # Вычисляем среднее (mu) и стандартное отклонение (sigma) приростов
    mu = diff_returns.mean()
    sigma = diff_returns.std()
    
    # Последнее значение цены как стартовое значение для генерации
    S0 = df["close"].iloc[-1]
    
    scenarios = []
    
    for _ in range(n_scenarios):
        # Генерируем изменения на основе нормального распределения
        changes = np.random.normal(mu, sigma, length)
        
        # Генерируем сценарий, накапливая изменения, начиная с последнего значения
        scenario = np.cumsum(changes) + S0
        scenarios.append(scenario)
    
    return scenarios



# Генерируем 5 
n_scenarios = 5
length = test_df.shape[0]
scenarios = generate_diff_scenarios(df, n_scenarios, length)

# Визуализация сценариев с использованием Plotly
fig = go.Figure()

# Добавляем оригинальную цену (последние значения) - красным цветом
fig.add_trace(go.Scatter(x=test_df['date'], y=test_df['close'],
                         mode='lines', name='Original Close',
                         line=dict(color='red')))

# Добавляем сгенерированные сценарии - синим цветом
for i, scenario in enumerate(scenarios):
    fig.add_trace(go.Scatter(x=test_df['date'], 
                             y=scenario, mode='lines', name=f'Scenario {i+1}',
                             line=dict(color='blue', width=1)))

# Настройка графика
fig.update_layout(title='Generated Price Scenarios Using Simple Returns',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  template='plotly_dark')

# Отображаем график
fig.show()